In [1]:
import numpy as np

urls_dict = np.load("../data/features_A.npy", allow_pickle=True).item() 

In [2]:
def sigmoid(x):
    return 1 / (1 + math.e ** -(x-20))

#IDF
import math
urls_sta = {}
urls_all_count = 0
url_max_depth = 0
for device in list(urls_dict.keys()):
    for url_tuple in urls_dict[device]:
        if url_tuple[0] not in list(urls_sta.keys()):
            urls_sta[url_tuple[0]] = 0
        urls_sta[url_tuple[0]] += url_tuple[1]
        urls_all_count += url_tuple[1]

        if len(url_tuple[0].split('.'))-1 > url_max_depth:
            url_max_depth = len(url_tuple[0].split('.'))-1

#split urls by domain level
# urls_domain_level_dict = {}
# urls_all_count_dm = 0
# for i in range(1,url_max_depth+1):
#     # url_tmp_dict = {}
#     for url in list(urls_sta.keys()):
#         if len(url.split('.'))-1 < i :
#             continue
#         url_split = '.'.join(url.split('.')[-i-1:])
#         if url_split not in list(urls_domain_level_dict.keys()):
#             urls_domain_level_dict[url_split] = 0
#         # accumulate urls with same dm lv
#         urls_domain_level_dict[url_split] += urls_sta[url]
#         urls_all_count_dm += urls_sta[url]


# another option: tfidf with urls (no split)
urls_domain_level_dict = {}
urls_all_count_dm = 0
for url in list(urls_sta.keys()):
    if url not in list(urls_domain_level_dict.keys()):
        urls_domain_level_dict[url] = 0
    urls_domain_level_dict[url] += urls_sta[url]
    urls_all_count_dm += urls_sta[url]
    

urls_idf = {}
for url in list(urls_domain_level_dict.keys()):
    urls_idf[url] = math.log(urls_all_count_dm / urls_domain_level_dict[url])

# sigmoid版本的tf_idf
urls_tfidf_dict = {}
idf_dict = {}
for device in urls_dict:
    if device not in urls_tfidf_dict:
        urls_tfidf_dict[device] = []
    # 统计url总数
    count = np.array([item[1] for item in urls_dict[device]]).sum()
    # (url, url in device, url not in device)
    urls_device_count_list = [(item[0], item[1], urls_domain_level_dict[item[0]] - item[1]) for item in urls_dict[device]]

    idf = [(item[0], item[1], sigmoid(urls_all_count_dm / (item[2] + 1))) for item in urls_device_count_list ]
    idf_dict[device] = idf
    tfidf_std_list = [(item[0], item[1], item[1]/count * sigmoid(urls_all_count_dm / (item[2] + 1)))  for item in urls_device_count_list]

    # 归一化
    tf_sum = np.array([item[2] for item in tfidf_std_list]).sum()
    tfidf_std_list = [(item[0], item[1], item[2]/tf_sum) for item in tfidf_std_list]
    # tfidf_std_list = [(item[0], item[1], item[1]/count) for item in tfidf_list]

    urls_tfidf_dict[device]  = tfidf_std_list
    



# 计算每一个url的tf_idf
# urls_tfidf_dict = {}
# for device in urls_dict:
#     if device not in urls_tfidf_dict:
#         urls_tfidf_dict[device] = []
#     # 统计url总数
#     count = np.array([item[1] for item in urls_dict[device]]).sum()
#     # 计算 tfidf
#     tfidf_list = [(item[0], item[1], item[1]/count*urls_idf[item[0]]) for item in urls_dict[device]]
#     # 归一化
#     tf_sum = np.array([item[2] for item in tfidf_list]).sum()
#     tfidf_std_list = [(item[0], item[1], item[2]/tf_sum) for item in tfidf_list]
#     # tfidf_std_list = [(item[0], item[1], item[1]/count) for item in tfidf_list]

#     urls_tfidf_dict[device] = tfidf_std_list

In [3]:
url_forest_all = {}

for device in urls_tfidf_dict.keys():
    #每一个设备一个森林
    if device not in list(url_forest_all.keys()):
        url_forest_all[device] = {'count_all':0}
    
    for url_tuple in urls_tfidf_dict[device]:
        #设备森林计数
        url_forest_all[device]['count_all'] += url_tuple[1]
        #二级域名
        sld = '.'.join(url_tuple[0].split('.')[-2:])
        if sld not in list(url_forest_all[device]):
            #新建一棵树
            url_forest_all[device][sld] = {}
            url_forest_all[device][sld]['count'] = 0 # url_tuple[1]
            url_forest_all[device][sld]['count_all_accm'] = 0 #url_tuple[1]
            url_forest_all[device][sld]['count_all'] = 0
            url_forest_all[device][sld]['urls'] = []
            
            # url_forest_all[device][sld]['children'] = {}
        # else:
        #     #树已存在,添加计数

        url_forest_all[device][sld]['count_all'] += url_tuple[1]
        url_forest_all[device][sld]['count_all_accm'] += url_tuple[1]
        #多级域名
        mlds = url_tuple[0].split('.')
        if len(mlds) <= 2:
            url_forest_all[device][sld]['count'] += url_tuple[1]
            url_forest_all[device][sld]['tfidf'] = url_tuple[2]
            if url_tuple[0] not in url_forest_all[device][sld]['urls']:
                url_forest_all[device][sld]['urls'].append(url_tuple[0])
            continue
        mlds.reverse()
        mlds = mlds[2:]

        current_node = url_forest_all[device][sld]
        #构建孩子结点
        for mld in mlds:
            #孩子结点不存在
            if mld not in list(current_node.keys()):
                current_node[mld] = {}
                current_node[mld]['count'] = 0 # url_tuple[1]
                current_node[mld]['count_all_accm'] = 0 # url_tuple[1]
            # else:
            #     #孩子结点存在，添加计数
            current_node[mld]['count_all_accm'] += url_tuple[1]
            current_node = current_node[mld]
        current_node['count'] += url_tuple[1]
        # current_node['count_all_accm'] += url_tuple[1]
        current_node['tfidf'] = url_tuple[2]
        if url_tuple[0] not in url_forest_all[device][sld]['urls']:
            url_forest_all[device][sld]['urls'].append(url_tuple[0])



In [4]:
def tree_var_aug(tree):
    #树权重更新
    # tree = url_forest_all['vivoV1816A']['com.cn']

    url_leaves = tree['urls']
    path_all_list = []

    #遍历每个url
    for url in url_leaves:
        sld = '.'.join(url.split('.')[-2:])
        mlds = url.split('.')
        mlds.reverse()
        mlds = mlds[2:]
        url_ld_list = [sld]
        url_ld_list.extend(mlds)

        #遍历每一个子域名
        path_per_list = [tree['count_all_accm']]
        current_node = tree
        for current_mld in url_ld_list[1:]:
            path_per_list.append(current_node[current_mld]['count_all_accm'])
            current_node = current_node[current_mld]
        path_all_list.append(path_per_list)
        # print(path_per_list)
        # print(url)

    #计算path的前后差值
    path_aug_list = []
    path_aug_times_list = []
    for path in path_all_list:
        tmp = []
        tmp_times = 1
        for i in range(len(path)-1):
            tmp.append(path[i]/path[i+1])
        for i in range(len(tmp)-1):
            tmp_times = tmp_times* tmp[i] * tmp[i+1]
        path_aug_times_list.append(math.log(tmp_times)+1)
        path_aug_list.append(tmp)

    aug_var = math.log(np.array(path_aug_times_list).sum())  + 1# prevent timing zero
    tree['aug_var'] = aug_var
    # print(tree)
    return tree

rej_list = ['count','count_all','count_all_accm','aug_var','major']
devices = list(url_forest_all.keys())
for device in devices:
    # print(device)
    for tree_name in url_forest_all[device]:
        # print(url_forest_all[device][tree_name])
        if tree_name in rej_list:
            
            continue
        url_forest_all[device][tree_name] = tree_var_aug(url_forest_all[device][tree_name])

In [5]:
#aug_var更新count后，更新tfidf
rej_list = ['count','count_all','count_all_accm','aug_var','major']
devices = list(url_forest_all.keys())
url_forest_all_device_regen_all = {}
for device in devices:
    
    url_forest_all_device_regen = url_forest_all[device].copy()

    for tree_name in url_forest_all[device]:
        if tree_name in rej_list:
            continue
        url_leaves = url_forest_all_device_regen[tree_name]['urls']

        #遍历每个url
        url_forest_all_device_regen[tree_name]['url_to_count_aug_tfidf'] = {}
        url_forest_all_device_regen[tree_name]['count_all_for_aug_tfidf'] = 0.0
        for url in url_leaves:
            sld = '.'.join(url.split('.')[-2:])
            mlds = url.split('.')
            mlds.reverse()
            mlds = mlds[2:]
            url_ld_list = [sld]
            url_ld_list.extend(mlds)

            #遍历每一个子域名
            current_node = url_forest_all_device_regen[tree_name]
            for current_mld in url_ld_list[1:]:
                current_node = current_node[current_mld]
            current_node['count'] = current_node['count']*url_forest_all_device_regen[tree_name]['aug_var']
            url_forest_all_device_regen[tree_name]['url_to_count_aug_tfidf'][url] = current_node['count']
            url_forest_all_device_regen[tree_name]['count_all_for_aug_tfidf'] += current_node['count']

    url_forest_all_device_regen_all[device] = url_forest_all_device_regen

#TF IDF
import math
urls_sta = {}
count_device_dict = {} # device: url count after aug
for device in devices:
    count_device_dict[device] = 0
    for tree_name in url_forest_all_device_regen_all[device]:
        if tree_name in rej_list:
            continue
        count_device_dict[device] += url_forest_all_device_regen_all[device][tree_name]['count_all_for_aug_tfidf']
        for url in url_forest_all_device_regen_all[device][tree_name]['url_to_count_aug_tfidf']:
            if url not in urls_sta:
                urls_sta[url] = 0.0 
            urls_sta[url] += url_forest_all_device_regen_all[device][tree_name]['url_to_count_aug_tfidf'][url]

def sigmoid(x):
    return 1 / (1 + math.e ** -(x-20))
#对比每条url在别的device中出现的次数，遍历取最小值作为idf
tfidf_val_for_std_dict = {} #归一化dict device:val_all
for device in devices:
    tfidf_val_for_std_dict[device] = 0.0
    for tree_name in url_forest_all_device_regen_all[device]:
        if tree_name in rej_list:
            continue
        url_forest_all_device_regen_all[device][tree_name]['url_to_tfidf_aug_min'] = {}
        for url in url_forest_all_device_regen_all[device][tree_name]['urls']:
            url_count_current_device = url
            url_count_current_device_count = url_forest_all_device_regen_all[device][tree_name]['url_to_count_aug_tfidf'][url]

            idf_list = []
            sld = '.'.join(url_count_current_device.split('.')[-2:])
            for device_other in devices:
                if device == device_other:
                    continue
                if sld in url_forest_all_device_regen_all[device_other]:
                    if url_count_current_device in url_forest_all_device_regen_all[device_other][sld]:
                        url_count_other_device_count = url_forest_all_device_regen_all[device_other][sld][url_count_current_device]
                    else:
                        url_count_other_device_count = 1 # prevent zero
                else:
                    url_count_other_device_count = 1 # prevent zero
                idf = url_count_current_device_count / url_count_other_device_count
                idf_list.append(idf)
            min_idf = sigmoid(np.array(idf_list).min())

            #计算每条url的tf-idf
            tfidf = min_idf * (url_count_current_device_count / count_device_dict[device])
            url_forest_all_device_regen_all[device][tree_name]['url_to_tfidf_aug_min'][url_count_current_device] = tfidf
            tfidf_val_for_std_dict[device] += tfidf

#标准化归一化
for device in devices:
    for tree_name in url_forest_all_device_regen_all[device]:
        if tree_name in rej_list:
            continue
        
        for url in url_forest_all_device_regen_all[device][tree_name]['url_to_tfidf_aug_min']:
            url_forest_all_device_regen_all[device][tree_name]['url_to_tfidf_aug_min'][url] = url_forest_all_device_regen_all[device][tree_name]['url_to_tfidf_aug_min'][url] / tfidf_val_for_std_dict[device]
            
                
                    


In [6]:
#统计每个device森林里的major树
rej_list = ['count','count_all','count_all_accm','aug_var','major']
for device in devices:
    major_value = 0
    url_forest_all_device_regen_all[device]['major'] = 0
    trees = url_forest_all_device_regen_all[device]
    for tree_name in trees:
        if tree_name in rej_list:
            continue
        if len(list(url_forest_all_device_regen_all[device][tree_name]['url_to_tfidf_aug_min'].keys())) > major_value:
            major_value = len(list(url_forest_all_device_regen_all[device][tree_name]['url_to_tfidf_aug_min'].keys()))
            url_forest_all_device_regen_all[device]['major'] = tree_name

In [10]:
import os,shutil
from scapy.all import *
from scapy.utils import PcapReader
import os
from subprocess import Popen, PIPE
from collections import Counter

def url_digger(file):

    def get_sec(item):
        return item[1]
    def get_fir(item):
        return item[0]
    
    def url_d(url):
        top_url = ['top','com','xyz','xin','vip','com','net','org','gov','edu','biz','name','info','mobi','pro','travel','club','museum','post','rec','asia','cc','cn','gov','info','net','nom','org','rec','store','web','io','local']
        # print(url)
        if url[-1] == '.':
            url = url[:-1]
        if url.split('.')[-1] in top_url:
            return True
        else:
            return False
    def Find(string): 
        url = re.findall('(?:[-\w.]|(?:%[\da-fA-F]{2}))+', string)
        return [item  for item in url if len(item.split('.')) >=2]
        
    packets=rdpcap(file)
    urls_all = []
    for pkt in packets:
        # print(pkt)
        pkt = str(pkt).replace('"', '')
#         pkt = pkt.replace('\\', '')
        url_per_pkt = [item for item in Find(pkt) if url_d(item)]
        tmp = []
        for item in url_per_pkt:
            if item[-1] == '.':
                tmp.append(item[:-1])
            else:
                tmp.append(item)
        if len(tmp) != 0 :
            urls_all.extend(tmp)
        # print(urls_all)
        # print(Find(pkt))
    # urls_all = sorted(urls_all, key=get_sec,reverse=True)

    return urls_all
    



### npy version

In [42]:
#测试函数
import numpy as np

#权重更新的验证函数
def match_per_pcap_update_val(urls_test_w, url_forest_all):
    #extract urls
    # print(urls_test_w)

    #matching
    devices =  list(url_forest_all.keys()) #['SumsungC5']#
    devices_value_dict = {}
    for device in devices:
        #提取森林里所有url
        url_val_all = {}
        for tree_name in url_forest_all[device]:
            if tree_name in rej_list:
                continue
            for url in url_forest_all[device][tree_name]['url_to_tfidf_aug_min']:
                url_val_all[url] = url_forest_all[device][tree_name]['url_to_tfidf_aug_min'][url]
        
        #查询url
        devices_value_dict[device] = [] #{device : []}


        for tuple_per in urls_test_w:

            if tuple_per[0] in url_val_all:
                devices_value_dict[device].append(url_val_all[tuple_per[0]])
            
            
    return devices_value_dict

#加入cover的权重更新的验证函数
def match_per_pcap_update_val_with_cover_tree(urls_test_w, url_forest_all,th):

    #matching
    devices =  list(url_forest_all.keys()) #['SumsungC5']#
    devices_value_dict = {}
    for device in devices:
        #提取森林里所有url
        url_val_all = {}
        for tree_name in url_forest_all[device]:
            if tree_name in rej_list:
                continue
            for url in url_forest_all[device][tree_name]['url_to_tfidf_aug_min']:
                url_val_all[url] = url_forest_all[device][tree_name]['url_to_tfidf_aug_min'][url]
        
        #查询url
        devices_value_dict[device] = [] #{device : []}

        #计算每棵树的覆盖率
        devices_cover_dict = {} #{device : {Tree1 : [], Tree2: []}}


        for tuple_per in urls_test_w:
            if tuple_per[0] in url_val_all:
                sld = '.'.join(tuple_per[0].split('.')[-2:])
                
                if sld not in devices_cover_dict.keys():
                    devices_cover_dict[sld] = []
                
                devices_cover_dict[sld].append(url_val_all[tuple_per[0]])

        #判断cover是否满足阈值
        for sld in devices_cover_dict:
            tree_all_value = np.array(list(url_forest_all[device][sld]['url_to_tfidf_aug_min'].values())).sum()
            if np.array(devices_cover_dict[sld]).sum()/ tree_all_value>= th:
                devices_value_dict[device].append(tree_all_value)
            else:
                devices_value_dict[device].extend(devices_cover_dict[sld])

                      
    return devices_value_dict

#加入major树的权重更新的验证函数
def match_per_pcap_update_val_with_major_tree(urls_test_w, url_forest_all, th = 0.5):

    #matching
    devices =  list(url_forest_all.keys()) #['SumsungC5']#
    devices_value_dict = {}
    for device in devices:
        #提取森林里所有url
        url_val_all = {}
        for tree_name in url_forest_all[device]:
            if tree_name in rej_list:
                continue
            for url in url_forest_all[device][tree_name]['url_to_tfidf_aug_min']:
                url_val_all[url] = url_forest_all[device][tree_name]['url_to_tfidf_aug_min'][url]
        
        #查询url
        devices_value_dict[device] = [] #{device : []}

        major_matched = False

        for tuple_per in urls_test_w:

            if tuple_per[0] in url_val_all:
                sld = '.'.join(tuple_per[0].split('.')[-2:])
                if sld == url_forest_all[device]['major']:
                    if not major_matched:
                        devices_value_dict[device].append(np.array(list(url_forest_all[device][sld]['url_to_tfidf_aug_min'].values())).sum())
                        major_matched = True
                else:
                    devices_value_dict[device].append(url_val_all[tuple_per[0]])
            
            
    return devices_value_dict

def cal_value_per_pcap(devices_value_dict):
    def getsec(item):
        return item[1]
    value_tuple_list = []
    #sum up each list
    for device in list(devices_value_dict.keys()):
        value_tuple_list.append((device, np.array(devices_value_dict[device]).sum()))
    value_tuple_list = sorted(value_tuple_list, key= getsec, reverse= True)

    return value_tuple_list

def clf_per_pcap(value_tuple_list, th = 0.0, level = "Model"):
    if level == "Model":
        if value_tuple_list[0][1] >= th:
            print(value_tuple_list[0][0])
            return 1,value_tuple_list[0][0]
        else:

            return 0,value_tuple_list[0][0]
    elif level == "Brand":
        if value_tuple_list[0][1] >= th:
            print(value_tuple_list[0][0])
            return 1,value_tuple_list[0][0]
        else:

            return 0,value_tuple_list[0][0]
    


urls_dict = np.load("./features_bg_15s.npy", allow_pickle=True).item() 
res_dict = {}
miss_dict = {}
for label in urls_dict:
    print(label)
    
    jud_list = []
    miss_list = []
        
    count = 1 

    for urls_test_w in urls_dict[label]:
        # devices_value_dict = match_per_pcap_update_val(urls_test_w, url_forest_all_device_regen_all)
        devices_value_dict = match_per_pcap_update_val_with_cover_tree(urls_test_w, url_forest_all_device_regen_all, th = 0.6)

        value_tuple_list = cal_value_per_pcap(devices_value_dict) 

        jud, pred = clf_per_pcap(value_tuple_list, label, th = 0.7, level = "Model") #Brand or Model level
        jud_list.append(jud)

        if jud == 0:
            miss_list.append(pred)

            
            
        if count % int(len(urls_dict[label]) / 10) == 0:
            print("{} / {}".format(count, len(urls_dict[label])))
        count += 1

    miss_dict[label] = miss_list
    res_dict[label] = jud_list

    

background
Xiaomi:Xiaomi2s
Xiaomi:Xiaomi2s
Xiaomi:Xiaomi2s
277 / 2776
554 / 2776
Xiaomi:Xiaomi2s
831 / 2776
Xiaomi:Xiaomi2s
1108 / 2776
1385 / 2776
Xiaomi:Xiaomi2s
1662 / 2776
1939 / 2776
2216 / 2776
2493 / 2776
Xiaomi:Xiaomi2s
2770 / 2776


In [43]:
acc_all = 0.0
all = 0.0
for models in res_dict:
    acc = np.array(res_dict[models]).sum()/len(res_dict[models])
    acc_all += np.array(res_dict[models]).sum()
    all += len(res_dict[models])
    print("{} : {}".format(models,acc))
print("{} : {}".format("Overall_ACC",acc_all/all))

background : 0.002521613832853026
Overall_ACC : 0.002521613832853026
